In [55]:
import sys
import pickle
import pandas as pd
from bitarray import bitarray

from nltk.tokenize import sent_tokenize
from coding import EliasCoding, NumberRepr
from retrieval import IRetrieval
from indexing import InvertedIndex
from preprocessing import Tokenization
from deduplication import deduplicate_data
from utils import get_object_size, get_time, load_dataset, load_file, save_file

### Проверка кодирования и декодирования Элиаса

In [2]:
#Проверка кодирования
for i in range(1, 3):
    num_repr = NumberRepr.Binary(i)
    elias_gamma_encode = EliasCoding.elias_gamma_encode(i)
    elias_gamma_decode = EliasCoding.elias_gamma_decode(elias_gamma_encode)
    elias_delta_encode = EliasCoding.elias_delta_encode(i)
    elias_delta_decode = EliasCoding.elias_delta_decode(elias_delta_encode)

    print(f"Бинарное представление числа {i}:", num_repr)
    print(f"Гамма кодирование Элиаса числа {i}:", elias_gamma_encode)
    print(f"Гамма декодирование Элиаса числа {i}:", elias_gamma_decode)
    print(f"Гамма кодирование Элиаса числа {i}:", elias_delta_encode)
    print(f"Дельта декодирование Элиаса числа {i}:", elias_delta_decode, "\n")

Бинарное представление числа 1: 1
Гамма кодирование Элиаса числа 1: 1
Гамма декодирование Элиаса числа 1: 1
Гамма кодирование Элиаса числа 1: 1
Дельта декодирование Элиаса числа 1: 1 

Бинарное представление числа 2: 10
Гамма кодирование Элиаса числа 2: 010
Гамма декодирование Элиаса числа 2: 2
Гамма кодирование Элиаса числа 2: 0100
Дельта декодирование Элиаса числа 2: 2 



In [3]:
#Проверка корректности декодирования строки 
#100, 1, 12, 1, 1, 12100
gamma_test = '00000011001001000110011000000000000010111101000100'
gamma_res = EliasCoding.decode_gamma_string(gamma_test)
print(gamma_res)
#1, 17, 8, 5, 1
delta_test = '100101000100100000011011'
delta_res = EliasCoding.decode_delta_string(delta_test)
print(delta_res)

[100, 1, 12, 1, 1, 12100]
[1, 17, 8, 5, 1]


### Подготовка данных

In [33]:
#Путь к файлу с данными
data_path = "/Users/petrovich/Desktop/my_code/data_collecting_projects/inverted_index/data/МГУ_2023-12-31.csv"
#Загрузка данных
docs = load_dataset(data_path)
print(len(docs), " – количество записей в корпусе")
print(len(set(docs)), " – количество уникальных записей в корпусе")

40069  – количество записей в корпусе
35061  – количество уникальных записей в корпусе


In [17]:
corpus_sentences_list = []
for doc in docs:  
    sentence_list = sent_tokenize(doc, language='russian')
    corpus_sentences_list.extend(sentence_list)
print(len(corpus_sentences_list), " – количество предложений в корпусе")
print(len(set(corpus_sentences_list)), " – количество уникальных предложений в корпусе")

659278  – количество предложений в корпусе
449575  – количество уникальных предложений в корпусе


### Дедупликация данных

In [62]:
copied_docs = docs.copy().astype("string")

In [56]:
deduplicated_docs = deduplicate_data(copied_docs)

In [57]:
deduplicated_docs

0        По страницам газет прошлых лет \n \n НА ВОЙНЕ ...
1        #новости@rsue.rinh | Студенты РГЭУ (РИНХ) стал...
2        Ведущие ученые МГУ имени Ломоносова провели ле...
3        🌺 Евгения ЛИСИЦЫНА в Московском университете 💐...
4        YouTalk — сервис психологической помощи - Подк...
                               ...                        
40012                                                40009
40013                                                40009
40014                                                40009
40029                                                32042
40037                                                 8550
Name: texts, Length: 45077, dtype: object

In [60]:
docs[32042]

'❗1-я подборка [club190320587|Научно-просветительского проекта НаукаPRO] об истории и археологии неандертальцев, и об уникальных находках эпохи среднего палеолита, сделанных на территории Ростовской и Волгоградской областей. \n \n• Неандертальцы среднего палеолита \n• Подвеска и инструменты неандертальцев из стоянки «Рожок» в Ростовской области \n• Украшение неандертальцев из стоянки среднего палеолита «Рожок» в Ростовской области \n• Орудия неандертальцев для копания земли с территории Ростовской области \n• Орудия неандертальцев для копания земли из Ростовской и Волгоградской областей \n• Как изучается средний палеолит \n• Достижения и упущения неандертальцев \n• Происхождение неандертальцев и кроманьонцев \n• Чем кроманьонцы отличались от неандертальцев \n• Детство в палеолите \n \nРассказывают: \n• Александр Колесник, археолог, доктор исторических наук, профессор исторического факультета Донецкого государственного университета. \n• Евгений Гиря, археолог, трасолог, кандидат историч

In [61]:
docs[40029]

'❗1-я подборка [club190320587|Научно-просветительского проекта НаукаPRO] об истории и археологии неандертальцев, и об уникальных находках эпохи среднего палеолита, сделанных на территории Ростовской и Волгоградской областей. \n \n• Неандертальцы среднего палеолита \n• Подвеска и инструменты неандертальцев из стоянки «Рожок» в Ростовской области \n• Украшение неандертальцев из стоянки среднего палеолита «Рожок» в Ростовской области \n• Орудия неандертальцев для копания земли с территории Ростовской области \n• Орудия неандертальцев для копания земли из Ростовской и Волгоградской областей \n• Как изучается средний палеолит \n• Достижения и упущения неандертальцев \n• Происхождение неандертальцев и кроманьонцев \n• Чем кроманьонцы отличались от неандертальцев \n• Детство в палеолите \n \nРассказывают: \n• Александр Колесник, археолог, доктор исторических наук, профессор исторического факультета Донецкого государственного университета. \n• Евгений Гиря, археолог, трасолог, кандидат историч

### Токенизация документов

In [ ]:
#Токенизация документов
tokenized_docs = Tokenization.tokenize_doc_corpus(docs)
#save_file(tokenized_docs, "tokenized_docs")

In [7]:
#Загрузка локального файла 
tokenized_docs = load_file("tokenized_docs")

### Построение инвертированного индекса

In [13]:
#700к без нормализации
inverted_index = InvertedIndex.create_inverted_index(tokenized_docs)
print(len(inverted_index.keys()), "– количество ключей в инвертированном индексе")

196138 – количество ключей в инвертированном индексе


### Оптимизация инвертированного индекса

In [16]:
#Инвертированный индекс после гамма кодирования Элиаса
elias_gamma_inverted_index = InvertedIndex.compress_inverted_index(inverted_index.copy(), mode="gamma")

In [17]:
#Инвертированный индекс после дельта кодирования Элиаса
elias_delta_inverted_index = InvertedIndex.compress_inverted_index(inverted_index.copy(), mode="delta")

### Сравнение размеров инвертированного индекса

In [9]:
print(round(get_object_size(inverted_index)), "КБ – размер инвертированного индекса до сжатия")
print(round(get_object_size(elias_gamma_inverted_index)), "КБ – размер инвертированного индекса после гамма кодирования Элиаса")
print(round(get_object_size(elias_delta_inverted_index)), "КБ – размер инвертированного индекса после дельта кодирования Элиаса")

84613 КБ – размер инвертированного индекса до сжатия
60394 КБ – размер инвертированного индекса после гамма кодирования Элиаса
56017 КБ – размер инвертированного индекса после дельта кодирования Элиаса


### Информационный поиск

In [10]:
#Поиск индексов релевантных документов
query = "Ректор МГУ"
basic_relevant_docs, basic_time = get_time(IRetrieval.get_relevant_doc_indices, query, inverted_index, compressed="")
gamma_relevant_docs, gamma_time = get_time(IRetrieval.get_relevant_doc_indices, query, elias_gamma_inverted_index, compressed="gamma")
delta_relevant_docs, delta_time = get_time(IRetrieval.get_relevant_doc_indices, query, elias_delta_inverted_index, compressed="delta")


In [11]:
print(len(basic_relevant_docs), f"найдено документов по OR запросу '{query}' за {basic_time}c обычным инвертированным индексом")
print(len(gamma_relevant_docs), f"найдено документов по OR запросу '{query}' за {gamma_time}c с использованием гамма кода Элиаса")
print(len(delta_relevant_docs), f"найдено документов по OR запросу '{query}' за {delta_time}c с использованием дельта кода Элиаса")

39224 найдено документов по OR запросу 'Ректор МГУ' за 0.004569c обычным инвертированным индексом
39224 найдено документов по OR запросу 'Ректор МГУ' за 0.86206c с использованием гамма кода Элиаса
39224 найдено документов по OR запросу 'Ректор МГУ' за 0.685798c с использованием дельта кода Элиаса
